In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('archive2'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def load_limited_data_from_directoryt(directory_path, limit):
    images = []
    labels = []
    image_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".tif") and "label" not in root:
                image_path = os.path.join(root, file)
                label_path = root.replace("train1/images", "train1/labels").replace(".tif", ".tif")
                label_path = os.path.join(label_path, file.replace(".tif", ".tif"))  # Append file name to directory path
                # print("Label path:", label_path)  # Debugging info
                if os.path.exists(label_path):
                    try:
                        image = io.imread(image_path, as_gray=True)
                        image = trans.resize(image, (1024,1024,1))  # Ensure the image has the correct shape
                        images.append(image)

                        label = io.imread(label_path, as_gray=True)
                        label = trans.resize(label, (1024,1024,1))  # Ensure the label has the correct shape
                        labels.append(label)

                        image_count += 1
                        print("Train image", image_count)
                        if image_count >= limit:
                            break
                    except Exception as e:
                        print("Error loading image or label:", e)
                else:
                    print("Label path does not exist:", label_path)
    return np.array(images), np.array(labels)

def load_limited_data_from_directoryv(directory_path, limit):
    images = []
    labels = []
    image_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".tif") and "label" not in root:
                image_path = os.path.join(root, file)
                label_path = root.replace("val/images", "val/labels").replace(".tif", ".tif")
                label_path = os.path.join(label_path, file.replace(".tif", ".tif"))  # Append file name to directory path
                # print("Label path:", label_path)  # Debugging info
                if os.path.exists(label_path):
                    image = io.imread(image_path, as_gray=True)
                    image = trans.resize(image, (1024,1024,1))  # Ensure the image has the correct shape
                    images.append(image)

                    label = io.imread(label_path, as_gray=True)
                    label = trans.resize(label, (1024,1024,1))  # Ensure the label has the correct shape
                    labels.append(label)

                    image_count += 1
                    print("val image", image_count)
                    if image_count >= limit:
                        break
    return np.array(images), np.array(labels)

def load_limited_data_from_directoryt2(directory_path, limit):
    images = []
    labels = []
    image_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".tif") and "label" not in root:
                image_path = os.path.join(root, file)
                label_path = root.replace("train2/images", "train2/labels").replace(".tif", ".tif")
                label_path = os.path.join(label_path, file.replace(".tif", ".tif"))  # Append file name to directory path
                # print("Label path:", label_path)  # Debugging info
                if os.path.exists(label_path):
                    try:
                        image = io.imread(image_path, as_gray=True)
                        image = trans.resize(image, (1024,1024,1))  # Ensure the image has the correct shape
                        images.append(image)

                        label = io.imread(label_path, as_gray=True)
                        label = trans.resize(label, (1024,1024,1))  # Ensure the label has the correct shape
                        labels.append(label)

                        image_count += 1
                        print("Train image", image_count)
                        if image_count >= limit:
                            break
                    except Exception as e:
                        print("Error loading image or label:", e)
                else:
                    print("Label path does not exist:", label_path)
    return np.array(images), np.array(labels)

In [ ]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

# Define paths to directories
train1_directory_path = 'archive2/train1/images'
train2_directory_path = 'archive2/train2/images'
val_directory_path = 'archive2/val/images'

# Load training and validation data for the first session
train_images1, train_labels1 = load_limited_data_from_directoryt(train1_directory_path, limit=1600)
val_images1, val_labels1 = load_limited_data_from_directoryv(val_directory_path, limit=800)

print("Train1 Images Shape:", train_images1.shape)
print("Train1 Labels Shape:", train_labels1.shape)
print("Validation Images Shape:", val_images1.shape)
print("Validation Labels Shape:", val_labels1.shape)

# Show the first training image and label
def show_image_and_label(images, labels, index):
    if index >= len(images) or index >= len(labels):
        print("Index out of bounds.")
        return
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(images[index].reshape(1024, 1024), cmap='gray')
    axs[0].set_title('Image')
    axs[1].imshow(labels[index].reshape(1024, 1024), cmap='gray')
    axs[1].set_title('Label')
    plt.show()

show_image_and_label(train_images1, train_labels1, index=50)
show_image_and_label(val_images1, val_labels1, index=90)

def resnet_model(input_size=(1024, 1024, 1)):
    inputs = Input(input_size)
    resnet_base = ResNet50(weights=None, include_top=False, input_tensor=inputs)

    # Add custom layers for segmentation
    x = resnet_base.output
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(16, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    outputs = Conv2D(1, 1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
model = resnet_model(input_size=(1024, 1024, 1))

# Load weights from the previously trained model (if any)
weights_path = 'resnet_weights.weights.h5'
if os.path.exists(weights_path):
    model.load_weights(weights_path)
    print("Weights loaded.")

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for the first session
history1 = model.fit(train_images1, train_labels1, validation_data=(val_images1, val_labels1), epochs=20, batch_size=4)

# Save weights after the first training session
model.save_weights(weights_path)

# Load training and validation data for the second session
train_images2, train_labels2 = load_limited_data_from_directoryt2(train2_directory_path, limit=1600)
val_images2, val_labels2 = load_limited_data_from_directoryv(val_directory_path, limit=800)

print("Train2 Images Shape:", train_images2.shape)
print("Train2 Labels Shape:", train_labels2.shape)
print("Validation Images Shape:", val_images2.shape)
print("Validation Labels Shape:", val_labels2.shape)

show_image_and_label(train_images2, train_labels2, index=10)
show_image_and_label(val_images2, val_labels2, index=30)

# Train the model for the second session
history2 = model.fit(train_images2, train_labels2, validation_data=(val_images2, val_labels2), epochs=20, batch_size=4)

# Save weights after the second training session
model.save_weights(weights_path)

# Save the trained model
model.save('res_roof_segmentation_model.keras')

# Plot combined training and validation loss
def plot_combined_loss(history1, history2):
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_combined_loss(history1, history2)

# Predict on the validation data
val_predictions1 = model.predict(val_images1)
val_predictions2 = model.predict(val_images2)

# Calculate metrics for both validation sets
def calculate_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = (y_pred.flatten() > 0.5).astype(int)  # Binarize predictions
    y_true = (y_true > 0.5).astype(int)  # Binarize true labels

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    return precision, recall, f1, cm

precision1, recall1, f1_1, cm1 = calculate_metrics(val_labels1, val_predictions1)
precision2, recall2, f1_2, cm2 = calculate_metrics(val_labels2, val_predictions2)

# Create a DataFrame to display the results
metrics_data = {
    'Session': ['Session 1', 'Session 2'],
    'Precision': [precision1, precision2],
    'Recall': [recall1, recall2],
    'F1 Score': [f1_1, f1_2],
    'Confusion Matrix': [cm1, cm2]
}

metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)

In [ ]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

# Define paths to directories
train1_directory_path = 'archive2/train1/images'
train2_directory_path = 'archive2/train2/images'
val_directory_path = 'archive2/val/images'

# Load training and validation data for the first session
train_images1, train_labels1 = load_limited_data_from_directoryt(train1_directory_path, limit=1600)
val_images1, val_labels1 = load_limited_data_from_directoryv(val_directory_path, limit=800)
# Load training and validation data for the second session
train_images2, train_labels2 = load_limited_data_from_directoryt2(train2_directory_path, limit=1600)

print("Train1 Images Shape:", train_images1.shape)
print("Train1 Labels Shape:", train_labels1.shape)
print("Validation Images Shape:", val_images1.shape)
print("Validation Labels Shape:", val_labels1.shape)
print("Train2 Images Shape:", train_images2.shape)
print("Train2 Labels Shape:", train_labels2.shape)

# Show the first training image and label
def show_image_and_label(images, labels, index):
    if index >= len(images) or index >= len(labels):
        print("Index out of bounds.")
        return
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(images[index].reshape(1024, 1024), cmap='gray')
    axs[0].set_title('Image')
    axs[1].imshow(labels[index].reshape(1024, 1024), cmap='gray')
    axs[1].set_title('Label')
    plt.show()

show_image_and_label(train_images1, train_labels1, index=50)
show_image_and_label(val_images1, val_labels1, index=90)
show_image_and_label(train_images2, train_labels2, index=10)
show_image_and_label(val_images1, val_labels1, index=30)
show_image_and_label(train_images1, train_labels1, index=20)
show_image_and_label(val_images1, val_labels1, index=20)
show_image_and_label(train_images2, train_labels2, index=45)
show_image_and_label(val_images1, val_labels1, index=45)

print("Res-Net Starts")

def resnet_model(input_size=(1024, 1024, 1)):
    inputs = Input(input_size)
    resnet_base = ResNet50(weights=None, include_top=False, input_tensor=inputs)

    # Add custom layers for segmentation
    x = resnet_base.output
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(16, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    outputs = Conv2D(1, 1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
model = resnet_model(input_size=(1024, 1024, 1))

# Load weights from the previously trained model (if any)
weights_path = 'resnet_weights.weights.h5'
if os.path.exists(weights_path):
    model.load_weights(weights_path)
    print("Weights loaded.")

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for the first session
history1 = model.fit(train_images1, train_labels1, validation_data=(val_images1, val_labels1), epochs=5, batch_size=4)

# Save weights after the first training session
model.save_weights(weights_path)


# Train the model for the second session
history2 = model.fit(train_images2, train_labels2, validation_data=(val_images1, val_labels1), epochs=5, batch_size=4)

# Save weights after the second training session
model.save_weights(weights_path)

# Save the trained model
model.save('res_roof_segmentation_model.keras')

# Plot combined training and validation loss
def plot_combined_loss(history1, history2):
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_combined_loss(history1, history2)

# Predict on the validation data
val_predictions1 = model.predict(val_images1)
val_predictions2 = model.predict(val_images1)

# Calculate metrics for both validation sets
def calculate_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = (y_pred.flatten() > 0.5).astype(int)  # Binarize predictions
    y_true = (y_true > 0.5).astype(int)  # Binarize true labels

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    return precision, recall, f1, cm

precision1, recall1, f1_1, cm1 = calculate_metrics(val_labels1, val_predictions1)
precision2, recall2, f1_2, cm2 = calculate_metrics(val_labels1, val_predictions2)

# Create a DataFrame to display the results
metrics_data = {
    'Session': ['Session 1', 'Session 2'],
    'Precision': [precision1, precision2],
    'Recall': [recall1, recall2],
    'F1 Score': [f1_1, f1_2],
    'Confusion Matrix': [cm1, cm2]
}

metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)

#---------------------------------------------------------------------U-Net---------------------------------------------------------------------------#

import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

print("U-Net Starts")

def unet(pretrained_weights = None,input_size = (1024,1024,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

# Function to display training image and label
def show_image_and_label(images, labels, index):
    if index >= len(images) or index >= len(labels):
        print("Index out of bounds.")
        return
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(images[index].reshape(1024,1024,1), cmap='gray')
    axs[0].set_title('Image')
    axs[1].imshow(labels[index].reshape(1024,1024,1), cmap='gray')
    axs[1].set_title('Label')
    plt.show()

# Function to plot combined training and validation loss
def plot_combined_loss(history1, history2):
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = (y_pred.flatten() > 0.5).astype(int)  # Binarize predictions
    y_true = (y_true > 0.5).astype(int)  # Binarize true labels

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    return precision, recall, f1, cm


# Instantiate the model
model = unet(input_size=(1024,1024,1))

# Load weights from the previously trained model (if any)
weights_path = 'unet_weights.weights.h5'
if os.path.exists(weights_path):
    model.load_weights(weights_path)
    print("Weights loaded.")

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for the first session
history1 = model.fit(train_images1, train_labels1, validation_data=(val_images1, val_labels1), epochs=5, batch_size=4)

# Save weights after the first training session
model.save_weights(weights_path)

# Train the model for the second session
history2 = model.fit(train_images2, train_labels2, validation_data=(val_images1, val_labels1), epochs=5, batch_size=4)


# Save weights after the second training session
model.save_weights(weights_path)

# Save the trained model
model.save('unet_roof_segmentation_model.keras')

# Plot combined training and validation loss
plot_combined_loss(history1, history2)


# Predict on the validation data
val_predictions1 = model.predict(val_images1)
val_predictions2 = model.predict(val_images1)

# Calculate metrics for both validation sets
precision1, recall1, f1_1, cm1 = calculate_metrics(val_labels1, val_predictions1)
precision2, recall2, f1_2, cm2 = calculate_metrics(val_labels1, val_predictions2)

# Create a DataFrame to display the results
metrics_data = {
    'Session': ['Session 1', 'Session 2'],
    'Precision': [precision1, precision2],
    'Recall': [recall1, recall2],
    'F1 Score': [f1_1, f1_2],
    'Confusion Matrix': [cm1, cm2]
}

metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)

#-------------------------------------------------------------------PSP-Net---------------------------------------------------------------------------#

print("PSP-Net Starts")

# Define the convolutional block and pyramid pooling module
def convolutional_block(input_tensor, filters, block_identifier):
    block_name = 'block_' + str(block_identifier) + '_'
    filter1, filter2, filter3 = filters
    skip_connection = input_tensor

    # Block A
    input_tensor = Conv2D(filters=filter1, kernel_size=(1, 1), dilation_rate=(1, 1),
                                 padding='same', kernel_initializer='he_normal', name=block_name + 'a')(input_tensor)
    input_tensor = BatchNormalization(name=block_name + 'batch_norm_a')(input_tensor)
    input_tensor = LeakyReLU(alpha=0.2, name=block_name + 'leakyrelu_a')(input_tensor)

    # Block B
    input_tensor = Conv2D(filters=filter2, kernel_size=(3, 3), dilation_rate=(2, 2),
                                 padding='same', kernel_initializer='he_normal', name=block_name + 'b')(input_tensor)
    input_tensor = BatchNormalization(name=block_name + 'batch_norm_b')(input_tensor)
    input_tensor = LeakyReLU(alpha=0.2, name=block_name + 'leakyrelu_b')(input_tensor)

    # Block C
    input_tensor = Conv2D(filters=filter3, kernel_size=(1, 1), dilation_rate=(1, 1),
                                 padding='same', kernel_initializer='he_normal', name=block_name + 'c')(input_tensor)
    input_tensor = BatchNormalization(name=block_name + 'batch_norm_c')(input_tensor)

    # Skip convolutional block for residual
    skip_connection = Conv2D(filters=filter3, kernel_size=(3, 3), padding='same', name=block_name + 'skip_conv')(skip_connection)
    skip_connection = BatchNormalization(name=block_name + 'batch_norm_skip_conv')(skip_connection)

    # Block C + Skip Convolution
    input_tensor = Add(name=block_name + 'add')([input_tensor, skip_connection])
    input_tensor = ReLU(name=block_name + 'relu')(input_tensor)
    return input_tensor

def base_convolutional_block(input_layer):
    base_result = convolutional_block(input_layer, [32, 32, 64], '1')
    base_result = convolutional_block(base_result, [64, 64, 128], '2')
    base_result = convolutional_block(base_result, [128, 128, 256], '3')
    return base_result

class PyramidPooling(tf.keras.layers.Layer):
    def __init__(self, pool_size, **kwargs):
        super(PyramidPooling, self).__init__(**kwargs)
        self.pool_size = pool_size
        self.conv = tf.keras.layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_normal')

    def call(self, inputs):
        shape = tf.shape(inputs)
        height, width = shape[1], shape[2]
        pooled = tf.keras.layers.AveragePooling2D(pool_size=self.pool_size)(inputs)
        pooled = self.conv(pooled)
        return tf.image.resize(pooled, size=(height, width), method='bilinear')

def pyramid_pooling_block(input_tensor):
    pyramid_block_1 = PyramidPooling(pool_size=(1, 1))(input_tensor)
    pyramid_block_2 = PyramidPooling(pool_size=(2, 2))(input_tensor)
    pyramid_block_3 = PyramidPooling(pool_size=(3, 3))(input_tensor)
    pyramid_block_4 = PyramidPooling(pool_size=(6, 6))(input_tensor)
    output_tensor = Concatenate()([input_tensor, pyramid_block_1, pyramid_block_2, pyramid_block_3, pyramid_block_4])
    return output_tensor

# Define the PSPNet model
def PSPNet(input_size=(256, 256, 1)):
    input_layer = Input(shape=input_size)
    base_layer = base_convolutional_block(input_layer)
    pyramid_block = pyramid_pooling_block(base_layer)
    output_layer = Conv2D(1, (1, 1), activation='sigmoid')(pyramid_block)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create PSPNet model instance
pspnet_model = PSPNet()

# Model summary
pspnet_model.summary()

# Load weights from the previously trained model (if any)
weights_path = 'pspnet_weights.weights.h5'
if os.path.exists(weights_path):
    model.load_weights(weights_path)
    print("Weights loaded.")

# Train the model using first batch of data
history1 = pspnet_model.fit(train_images1, train_labels1, epochs=1, validation_data=(val_images1, val_labels1))

# Save weights after the first training session
model.save_weights(weights_path)


# Train the model using the second batch of data
history2 = pspnet_model.fit(train_images2, train_labels2, epochs=1, validation_data=(val_images1, val_labels1))

# Save weights after the second training session
model.save_weights(weights_path)

# Save the trained model
model.save('psp_roof_segmentation_model.keras')

# Plot combined training and validation loss
plot_combined_loss(history1, history2)

# Evaluate the model on validation data
val_predictions = pspnet_model.predict(val_images1)
precision, recall, f1, cm = calculate_metrics(val_labels1, val_predictions)

# Print the metrics in a box format
print(f"{'Metric':<10} {'Value':<10}")
print(f"{'-'*20}")
print(f"{'Precision':<10} {precision:<10.4f}")
print(f"{'Recall':<10} {recall:<10.4f}")
print(f"{'F1 Score':<10} {f1:<10.4f}")
print(f"{'-'*20}")
print(f"Confusion Matrix:\n{cm}")

In [ ]:
!pip install --upgrade tensorflow keras
# remember to run this if the comp has been shut down before